In [ ]:
# ResNet-50 Code
# Importing Libraries
import keras 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.layers as layers

# Dimensions of our images.
img_width, img_height = 256,256


# Input and training parameters
train_data_dir = '/kaggle/input/hne/hfolds/f1/train/'
validation_data_dir = '/kaggle/input/hne/hfolds/f1/validation/'
nb_train_samples = 2274
nb_validation_samples = 750
epochs = 50
batch_size = 12
chanDim = -1


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [ ]:
# Deining the Identity and convolution blocks
def identity_block(input_tensor, kernel_size, num_filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    bn_axis = 3

    filters1, filters2, filters3 = num_filters
  
   
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=(1, 1),padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1), padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    # Skip Connection
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

np.random.seed(0)
X = np.random.randn(3, 4, 4, 6)
print('original data shape:', X.shape)
  

stage = 1
block = 'a'
inputs = layers.Input(shape=X.shape[1:])

outputs = identity_block(inputs, kernel_size=2, num_filters=[2, 4, 6], stage=stage, block=block)

model = Model(inputs=inputs, outputs=outputs)
prediction = model.predict(X)
print('identity block output shape:', prediction.shape)
prediction[1, 1, 0]

def conv_block(input_tensor, kernel_size, num_filters, stage, block, strides=(2, 2)):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    bn_axis = 3

    filters1, filters2, filters3 = num_filters
  
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=strides,padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1), padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # we resize the input so its dimension will match the output dimension
    # of the main path
    shortcut = layers.Conv2D(filters3, kernel_size=(1, 1), strides=strides,padding='valid', name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut) 

    # Shortcut to the main path before activation
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


In [ ]:
def ResNet(input_shape, n_classes):
    img_input = layers.Input(shape=input_shape)
    
    bn_axis = 3
    
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),strides=(2, 2),padding='valid', name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    x = AveragePooling2D((2,2), name="avg_pool")(x)
    
    x = Flatten()(x)
    
   
    
    img_output = layers.Dense(1, activation='sigmoid', name='fc' + str(4))(x)

    model = Model(inputs=img_input, outputs=img_output, name='resnet')
    return model


model = ResNet(input_shape,2)
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

with open('resnet50.json', 'w') as f:
    json.dump(str(history.history), f)
    
# Plotting Accuracy Curve
history_df = pd.DataFrame(history.history)
history_df[['accuracy', 'val_accuracy']].plot()
plt.title('Training accuracy vs Validation accuracy of ResNet-50 for HnE')
plt.ylabel('Accuracy')
plt.xlabel('No of epochs')

In [ ]:
# Plotting Loss Curve 
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
plt.title('Training loss vs Validation loss of ResNet-50 for HnE')
plt.ylabel('Loss')
plt.xlabel('No of epochs')

In [ ]:
# Testing
test_data_dir = '/kaggle/input/hne/hfolds/f1/test'
test_data_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=1,
    class_mode="binary", 
    shuffle=False)

In [ ]:
# Saving Model weights
MODEL_FILE = "resnet50.h5"
model.save_weights(MODEL_FILE)

# Getting predictions for test images
probabilities = model.predict_generator(test_generator, 762)
print(probabilities)

In [ ]:
# Thresholding prediction probabilities at 0.5
prob=[]
print(len(probabilities))
for i in range(0, 762):
    if (probabilities[i][0] > 0.5):
        prob.append(1)
    else:
        prob.append(0)
    
print(prob)

In [ ]:
# Generating true labels
labels=[]
# Benign
for i in range(0, 516):
    labels.append(0)
    
# Malignant
for i in range(0, 246):
    labels.append(1)

In [ ]:
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

# Print the confusion Matrix
cm = confusion_matrix(labels, prob)
print(cm)